<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>

# 2.0 모델 호스팅

이 노트북에서는 배포 성능을 개선하기 위해 Triton Server를 최적화하는 전략을 배웁니다.


**[2.1 동시 모델 실행](#2.1-동시-모델-실행)**<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.1.1 연습: 사용 고려 사항](#2.1.1-연습:-사용-고려-사항)<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.1.2 구현](#2.1.2-구현)<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.1.3 연습: 여러 인스턴스 그룹 구성](#2.1.3-연습:-여러-인스턴스-그룹-구성)<br>
**[2.2 스케줄링 전략](#2.2-스케줄링-전략)**<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.2.1 스테이트리스 추론](#2.2.1-스테이트리스-추론)<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.2.2 스테이트풀 추론](#2.2.2-스테이트풀-추론)<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.2.3 파이프라인/앙상블](#2.2.3-파이프라인/앙상블)<br>
**[2.3 동적 배치](#2.3-동적-배치)**<br>
&nbsp; &nbsp; &nbsp; &nbsp; [2.3.1 연습: 동적 배치 구현](#2.3.1-연습:-동적-배치-구현)<br>

지금까지 고객 요청을 서버에 도착한 순서대로 순차적으로 실행했으며 서버에 도착하는 모든 요청에 크기 8의 정적 배치를 사용했습니다. 그 결과 GPU가 거의 활용되지 않은 상태로 남았을 뿐 아니라 서버에서 수신한 응답의 지연 시간에 큰 영향을 미쳤습니다. 이는 드문 상황이 아닙니다. 대량의 데이터를 배치로 처리하는 애플리케이션을 개발하지 않으면 사용자 애플리케이션에서 개별 추론 요청을 보낼 가능성이 높으므로 저활용 상태가 훨씬 더 심해질 수 있습니다. 이전 노트북에서 살펴본 것처럼 모델 최적화는 모델 실행을 가속화하는 데 상당한 도움이 됩니다.  그러나 서빙이 단순하게 구현될 경우 추론 워크로드의 특성으로 인해 GPU 저활용 상태로 이어진다는 사실은 변하지 않습니다.

NVIDIA Triton가 같은 추론 서버는 GPU 활용률을 높이고 요청 지연 시간을 개선할 수 있는 다양한 기능을 구현합니다. 이 수업에서는 다음 세 가지를 설명하겠습니다.<br/>
- <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/user_guide/architecture.html#concurrent-model-execution">동시 모델 실행</a></br>
- <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/user_guide/model_configuration.html#scheduling-and-batching">스케줄링</a> <br/>
- <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/user_guide/model_configuration.html#dynamic-batcher">동적 배치</a> <br/>


모델 추론 성능 개선에 도움이 될 수 있는 메커니즘 및 구성에 대한 자세한 내용은 <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/getting_started/quickstart.html">Triton 문서</a> 및 해당 <a href="https://github.com/NVIDIA/triton-inference-server">소스 코드</a>를 참조하십시오.

# 2.1 동시 모델 실행
Triton 아키텍처를 사용하면 여러 모델 및/또는 동일한 모델의 여러 인스턴스를 단일 GPU에서 병렬로 실행할 수 있습니다. 다음 그림은 두 가지 모델 `model0` 및 `model1`을 실행한 예를 보여줍니다. Triton이 현재 어떤 요청도 처리하지 않고 있다고 가정할 때 각 모델에 대해 하나씩 2개의 요청이 동시에 도착할 경우 Triton이 즉시 두 요청을 GPU에 예약하고 GPU의 하드웨어 스케줄러가 두 연산을 병렬로 처리하기 시작합니다. </br>

<img src="images/multi_model_exec.png"/><br/>

#### 기본 동작

기본적으로 동일한 모델에 대해 여러 개의 요청이 동시에 도착할 경우 Triton은 다음 그림과 같이 GPU에서 한 번에 한 개의 요청만 예약하여 요청 실행을 직렬화합니다.

<img src="images/multi_model_serial_exec.png"/><br/>

Triton은 각 모델이 모델에서 허용해야 하는 병렬 실행 수를 지정할 수 있는 인스턴스-그룹 기능을 제공합니다. 그렇게 활성화된 병렬 실행을 각각 *실행 인스턴스*라고 합니다. 기본적으로 Triton은 각 모델에 단일 실행 인스턴스를 제공하며, 이는 위 그림에 표시된 것처럼 한 번에 모델의 하나의 실행만 진행할 수 있음을 의미합니다. 

#### 인스턴스 그룹
*인스턴스-그룹* 설정을 사용하여 모델에 대한 실행 인스턴스의 수를 늘릴 수 있습니다. 다음 그림은 `model1`이 3개의 실행 인스턴스를 허용하도록 구성된 경우 모델 실행을 보여줍니다. 그림과 같이 처음 3개의 `model1` 추론 요청이 GPU에서 즉시 병렬로 실행됩니다. 네 번째 `model1` 추론 요청은 처음 3개의 실행 중 하나가 완료될 때까지 기다려야 시작됩니다.

<img src="images/multi_model_parallel_exec.png"/><br/>


## 2.1.1 연습: 사용 고려 사항

대부분의 모델의 경우 가장 큰 성능 개선을 제공하는 Triton 기능은 *동적 배치*입니다. 여러 인스턴스 실행을 설정하는 것과 비교할 때 동적 배치의 주요 이점은 다음과 같습니다.
- 모델 매개변수 저장을 위한 오버헤드 없음
- GPU 메모리에서 모델 매개변수 가져오기와 관련된 오버헤드 없음
- GPU 리소스를 더 잘 활용할 수 있음

여러 모델 실행의 구성을 살펴보기 전에 단일 인스턴스를 사용해 모델을 다시 실행하고 GPU의 리소스 활용률을 관찰해 보겠습니다. <br>


#### 연습 단계
1. JupyterLab 시작 페이지에서 터미널 창을 시작합니다.  새 시작 페이지를 열어야 할 경우 왼쪽 사이드바 메뉴에서 '+' 아이콘을 클릭합니다. 그런 다음 드래그 앤 드롭 작업을 사용하여 터미널을 더 잘 볼 수 있도록 하위 창 구성으로 이동할 수 있습니다.
2. 성능 도구를 실행하기 전에 터미널에서 다음 명령을 실행합니다.<br>

```
watch -n0.5 nvidia-smi
```
    다음과 같은 출력이 표시되어야 합니다.
<img src="images/NVIDIASMI.png" style="position:relative; left:30px;" width=800/>

3. 이전 노트북에서 사용한 것과 동일한 벤치마크를 실행하되 배치 크기를 1로 줄이고 <code>nvidia-smi</code> 출력을 다시 관찰합니다.  메모리 소비 및 GPU 활용률에 특히 주목하십시오.

In [1]:
# Set the server hostname and check it - you should get a message that "Triton Server is ready!"
tritonServerHostName = "triton"
!./utilities/wait_for_triton_server.sh {tritonServerHostName}

Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!


In [2]:
# Load the previous configuration.
modelVersion = "1"
precision = "fp32"
batchSize = "8"
maxLatency = "500"
maxClientThreads = "10"
maxConcurrency = "2"
dockerBridge = "host"
resultsFolderName = "1"
profilingData = "utilities/profiling_data_int64"
measurement_request_count = 50
percentile_stability = 85
stability_percentage = 50

In [3]:
%%time
# Update configuration parameters and run profiler.
modelName = "bertQA-onnx-trt-fp16"
maxConcurrency= "10"
batchSize="1"
print("Running: "+modelName)

!./utilities/run_perf_analyzer_local.sh \
                    {modelName} \
                    {modelVersion} \
                    {precision} \
                    {batchSize} \
                    {maxLatency} \
                    {maxClientThreads} \
                    {maxConcurrency} \
                    {tritonServerHostName} \
                    {dockerBridge} \
                    {resultsFolderName} \
                    {profilingData} \
                    {measurement_request_count} \
                    {percentile_stability} \
                    {stability_percentage}

Running: bertQA-onnx-trt-fp16
Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 1
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 50
  Latency limit: 500 msec
  Concurrency limit: 10 concurrent requests
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Pass [1] throughput: 55.9789 infer/sec. p85 latency: 17721 usec
  Pass [2] throughput: 56.9691 infer/sec. p85 latency: 17721 usec
  Pass [3] throughput: 55.9739 infer/sec. p85 latency: 17736 usec
  Client: 
    Request count: 169
    Throughput: 56.3073 infer/sec
    Avg client overhead: 0.01%
    Avg send request rate: 56.64 infer/sec
    p50 latency: 17673 usec
    p85 latency: 17724 usec
    p90 latency: 17737 usec
    p95 latency: 17758 usec
    p99 latency: 17820 usec
    Avg gRPC time: 17675 usec (marshal 2 usec + response wait 17672 usec + unm

다음과 비슷한 활용률이 관찰되었을 것입니다.<br/>
<img src="images/NVIDIASMI2.png" width=800/><br/>

인스턴스 그룹의 수를 늘린 결과로 큰 속도 증가가 관찰될 것이라고 생각하십니까?<br>
강사와 논의하십시오.

## 2.1.2 구현
동시 실행을 활성화하는 방법과 이것이 모델 성능에 미치는 영향을 살펴보겠습니다. 다음 코드 셀을 실행하여 모델을 ONNX 형식으로 내보냅니다.

In [4]:
modelName = "bertQA-onnx-conexec"
exportFormat = "onnx"

In [5]:
!python ./deployer/deployer.py \
    --{exportFormat} \
    --save-dir ./candidatemodels \
    --triton-model-name {modelName} \
    --triton-model-version 1 \
    --triton-max-batch-size 8 \
    --triton-dyn-batching-delay 0 \
    --triton-engine-count 1 \
    -- --checkpoint ./data/bert_qa.pt \
    --config_file ./bert_config.json \
    --vocab_file ./vocab \
    --predict_file ./squad/v1.1/dev-v1.1.json \
    --do_lower_case \
    --batch_size=8 

deploying model bertQA-onnx-conexec in format onnxruntime_onnx
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__2
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each d

In [6]:
!ls -alh ./candidatemodels/bertQA-onnx-conexec

total 16K
drwxr-xr-x 3 root root 4.0K Mar  4 06:19 .
drwxr-xr-x 3 root root 4.0K Mar  4 06:18 ..
drwxr-xr-x 2 root root 4.0K Mar  4 06:18 1
-rw-r--r-- 1 root root  569 Mar  4 06:19 config.pbtxt


## 2.1.3 연습: 여러 인스턴스 그룹 구성
여러 인스턴스를 지정하려면 "config.pbtxt" 구성 파일에 있는 `instance_group` 섹션에서 "개수" 값을 '1'에서 더 큰 수로 변경해야 합니다. 


```
    instance_group [
    {
        count: 2
        kind: KIND_GPU
        gpus: [ 0 ]
    }
]
```

#### 연습 단계:
1. 방금 만든 `bertQA-onnx-conexec` 배포에서 [config.pbtxt](candidatemodels/bertQA-onnx-conexec/config.pbtxt)를 수정하여 BERT 기반 질문 답변 모델의 두 인스턴스를 지정합니다. 파일의 끝에서 기본 instance_group 블록을 찾아야 합니다. 개수 변수를 1에서 2로 변경합니다.  (필요에 따라 [솔루션](solutions/ex-2-1-3_config.pbtxt) 참조)
2. 공정하게 비교하려면 `optimization` 블록 내에 `execution_accelerators` 블록을 추가해 TensorRT도 활성화합니다.

```text
optimization {
   execution_accelerators {
      gpu_execution_accelerator : [ {
         name : "tensorrt"
         parameters { key: "precision_mode" value: "FP16" }
      }]
   }
cuda { graphs: 0 }
}
```

3. 변경 사항을 저장한 후(주 메뉴: 파일 -> 파일 저장), 다음 명령을 실행하여 모델을 Triton으로 이동합니다.

In [ ]:
# quick fix!
!cp solutions/ex-2-1-3_config.pbtxt candidatemodels/bertQA-onnx-conexec/config.pbtxt

In [8]:
!mv ./candidatemodels/bertQA-onnx-conexec model_repository/

mv: cannot stat './candidatemodels/bertQA-onnx-conexec': No such file or directory


4. 모델에 대해 표준 스트레스 테스트를 실행합니다. 이를 단일 인스턴스 실행과 비교하십시오.<br>
   처리량이 변경되었습니까?<br>
   지연 시간이 변경되었습니까?

In [9]:
%%time
modelName = "bertQA-onnx-conexec"
maxConcurrency= "10"
batchSize="1"
print("Running: "+modelName)

!./utilities/run_perf_analyzer_local.sh \
                    {modelName} \
                    {modelVersion} \
                    {precision} \
                    {batchSize} \
                    {maxLatency} \
                    {maxClientThreads} \
                    {maxConcurrency} \
                    {tritonServerHostName} \
                    {dockerBridge} \
                    {resultsFolderName} \
                    {profilingData} \
                    {measurement_request_count} \
                    {percentile_stability} \
                    {stability_percentage}

Running: bertQA-onnx-conexec
Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 1
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 50
  Latency limit: 500 msec
  Concurrency limit: 10 concurrent requests
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Pass [1] throughput: 1.1772 infer/sec. p85 latency: 6181 usec
  Pass [2] throughput: 163.923 infer/sec. p85 latency: 6181 usec
  Pass [3] throughput: 163.926 infer/sec. p85 latency: 6149 usec
  Pass [4] throughput: 163.925 infer/sec. p85 latency: 6184 usec
  Client: 
    Request count: 492
    Throughput: 163.925 infer/sec
    Avg client overhead: 0.04%
    Avg send request rate: 163.92 infer/sec
    p50 latency: 6085 usec
    p85 latency: 6171 usec
    p90 latency: 6189 usec
    p95 latency: 6215 usec
    p99 latency: 6247 usec
    Avg gRPC time: 609

계속하기 전에 Triton 모델 리포지토리에서 일부 모델을 이동하여 GPU 메모리를 확보하겠습니다.  다음 3개 모델을 제거한 후 `bertQA-torchscript` 모델만 남아야 합니다.

In [10]:
# Remove models from the inference server by removing them from the model_repository
!mv -f /dli/task/model_repository/bertQA-onnx /dli/task/candidatemodels/
!mv -f /dli/task/model_repository/bertQA-onnx-conexec /dli/task/candidatemodels/
!mv -f /dli/task/model_repository/bertQA-onnx-trt-fp16 /dli/task/candidatemodels/

# List remaining models on the inference server
!ls /dli/task/model_repository

bertQA-torchscript


# 2.2 스케줄링 전략
Triton은 개별 추론 요청에서 입력 배치를 지정할 수 있도록 허용하여 배치 추론을 지원합니다. 입력 배치 추론은 동시에 수행되며, 이는 추론 처리량을 크게 늘릴 수 있으므로 GPU에 특히 중요합니다. 많은 사용 사례에서 개별 추론 요청은 배치 처리되지 않으므로 배치의 처리량 이점을 얻을 수 없습니다. <br/>

추론 서버에는 여러 가지 다양한 모델 유형과 사용 사례를 지원하는 여러 스케줄링 및 배치 알고리즘이 포함되어 있습니다. 스케줄러/배처 선택은 여러 가지 요인에 따라 결정되며 핵심 요인은 다음과 같습니다.
- 추론 워크로드의 스테이트풀/스테이트리스 특성
- 애플리케이션이 격리되어 처리되는 모델로 구성되어 있는지 여부 또는 더 복잡한 파이프라인/앙상블이 사용되는지 여부

## 2.2.1 스테이트리스 추론

이 수업에서처럼 스테이트리스 추론을 다룰 경우 스케줄링과 관련해서 두 가지 주요 옵션을 사용할 수 있습니다. 첫 번째 옵션은 추론에 할당된 모든 인스턴스에 요청을 분산하는 기본 스케줄러입니다. 이는 추론 워크로드의 구조가 잘 이해되고 추론이 규칙적인 배치 크기 및 시간 간격으로 발생하는 경우에 선호되는 옵션입니다.

두 번째 옵션은 개별 요청을 결합하여 기본 배처와 유사하게 대규모 배치를 인스턴스 전체에 분산하는 동적 배치입니다. 이 특별 옵션은 수업의 다음 섹션에서 설명하겠습니다.

## 2.2.2 스테이트풀 추론

스테이트풀 모델(또는 스테이트풀 맞춤형 백엔드)은 추론 요청 간에 상태를 유지합니다. 이 모델은 여러 추론 요청이 동일한 모델 인스턴스로 라우팅되어야 하는 추론 시퀀스를 형성하므로 모델에서 유지하는 상태가 올바르게 업데이트됩니다. 뿐만 아니라 이 모델은 Triton이 예를 들어 시퀀스 시작을 나타내는 제어 신호를 제공하도록 요구할 수도 있습니다.

시퀀스 배처는 동일한 모델 인스턴스로 라우팅되는 시퀀스를 배치 처리하는 방식을 결정할 때 2개의 스케줄링 전략 중 하나를 사용할 수 있습니다. 이러한 전략은 다이렉트(Direct) 스케줄링 전략과 올디스트(Oldest) 스케줄링 전략입니다.

다이렉트 스케줄링 전략을 사용하면 시퀀스 배처가 시퀀스의 모든 추론 요청이 동일한 모델 인스턴스로 라우팅되게 할 뿐 아니라 각 시퀀스가 모델 인스턴스 내의 전용 배치 슬롯으로 라우팅되게 합니다. 이 전략은 모델이 각 배치 슬롯의 상태를 유지할 경우에 필요하며, 주어진 시퀀스의 모든 추론 요청이 동일한 슬롯으로 라우팅되므로 상태가 올바르게 업데이트됩니다.

올디스트 스케줄링 전략을 사용하면 시퀀스 배처가 시퀀스의 모든 추론 요청이 동일한 모델 인스턴스로 라우팅된 다음 동적 배처를 사용하여 다양한 시퀀스의 여러 추론을 함께 추론을 수행하는 배치로 한꺼번에 처리하도록 합니다.

## 2.2.3 파이프라인/앙상블

앙상블 모델은 하나 이상의 모델의 파이프라인과 그러한 모델 간의 입력 및 출력 텐서 연결을 나타냅니다. 앙상블 모델은 "데이터 사전 처리 -> 추론 -> 데이터 사후 처리"와 같은 여러 모델이 관련된 절차를 캡슐화하는 데 사용되도록 고안되었습니다. 이 목적으로 앙상블 모델을 사용할 경우 중간 텐서를 전송하는 데 따른 오버헤드를 피하고 Triton으로 전송되어야 하는 요청의 수를 최소화할 수 있습니다. 앙상블 파이프라인의 예가 아래에 나와 있습니다. <br/>

<img src="images/ensemble_example0.png"/>

앙상블 내에서 모델이 사용하는 스케줄러에 관계없이 앙상블 스케줄러가 앙상블 모델에 사용되어야 합니다. 앙상블 스케줄러와 관련해서 앙상블 모델은 실제 모델이 아닙니다. 대신, 앙상블 모델은 앙상블 내의 모델 간의 데이터 흐름을 단계로 지정합니다. 스케줄러는 각 단계에서 출력 텐서를 수집하고 이를 지정에 따라 다른 단계의 입력 텐서로 제공합니다. 그럼에도 불구하고 앙상블 모델은 여전히 외부에서 봤을 때 단일 모델로 보여집니다.

Triton 스케줄링에 대한 자세한 내용은 <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/user_guide/architecture.html#stateless-models">문서의 다음 섹션</a>에서 확인할 수 있습니다. 이 수업에서는 Triton의 가장 강력한 기능 중 하나인 *동적 배치*를 더 중점적으로 살펴보겠습니다.

# 2.3 동적 배치
동적 배치는 Triton의 한 기능으로, 배치가 동적으로 생성되도록 서버에서 추론 요청을 결합할 수 있어 결과적으로 처리량이 늘어납니다.

모델 인스턴스를 추론에 사용할 수 있게 될 경우 동적 배처가 스케줄러에서 사용할 수 있는 요청에서 배치를 만들려고 시도합니다. 요청이 수신된 순서로 배치에 추가됩니다. 동적 배처가 선호하는 크기의 배치를 형성할 수 있는 경우 가능한 한 가장 큰 선호하는 크기의 배치를 만들고 추론을 위해 전송합니다. 동적 배처가 선호하는 크기의 배치를 형성할 수 없는 경우에는 모델에서 허용하는 최대 배치 크기보다 작은 가능한 한 가장 큰 배치를 전송합니다. 

동적 배처는 다른 요청이 동적 배치에 참여할 수 있도록 스케줄러에서 제한된 시간 동안 요청을 지연시킬 수 있게 구성할 수 있습니다. 예를 들어, 다음 구성에서는 요청에 대해 최대 100마이크로초의 지연 시간을 설정합니다.

```
dynamic_batching {
  preferred_batch_size: [ 4, 8 ]
  max_queue_delay_microseconds: 100
}
```


## 2.3.1 연습: 동적 배치 구현


다시 ONNX 모델을 내보내는 것으로 시작해 보겠습니다.

In [11]:
modelName = "bertQA-onnx-trt-dynbatch"
exportFormat = "onnx"

In [12]:
!python ./deployer/deployer.py \
    --{exportFormat} \
    --save-dir ./candidatemodels \
    --triton-model-name {modelName} \
    --triton-model-version 1 \
    --triton-max-batch-size 8 \
    --triton-dyn-batching-delay 0 \
    --triton-engine-count 1 \
    -- --checkpoint ./data/bert_qa.pt \
    --config_file ./bert_config.json \
    --vocab_file ./vocab \
    --predict_file ./squad/v1.1/dev-v1.1.json \
    --do_lower_case \
    --batch_size=8

deploying model bertQA-onnx-trt-dynbatch in format onnxruntime_onnx
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__0
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input__2
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to e

#### 연습 단계
1. 예제 스니펫을 사용하여 동적 배치의 [config.pbtxt](candidatemodels/bertQA-onnx-trt-dynbatch/config.pbtxt)를 수정합니다. 

    ```
    dynamic_batching {
      preferred_batch_size: [ 4, 8 ]
      max_queue_delay_microseconds: 100
    }
    ```
    
2. 최적화 블록에서 TensorRT를 활성화합니다.

    ```
    optimization {
       execution_accelerators {
          gpu_execution_accelerator : [ {
             name : "tensorrt"
             parameters { key: "precision_mode" value: "FP16" }
          }]
       }
    cuda { graphs: 0 }
    }
    ```
3. 저장되면 모델을 Triton 모델 리포지토리로 이동하고 다음 셀을 실행하여 성능 유틸리티를 실행합니다. (필요한 경우 [솔루션](solutions/ex-2-3-1_config.pbtxt) 참조)

In [13]:
# quick fix!
!cp solutions/ex-2-3-1_config.pbtxt candidatemodels/bertQA-onnx-trt-dynbatch/config.pbtxt

In [14]:
!mv ./candidatemodels/bertQA-onnx-trt-dynbatch model_repository/

In [15]:
%%time
# warm up model with some inferences for faster analysis  (takes about 5 minutes)
modelName = "bertQA-onnx-trt-dynbatch"
batchSize = 8
!./utilities/run_warmup.sh {modelName} {batchSize}
batchSize = 4
!./utilities/run_warmup.sh {modelName} {batchSize}
batchSize = 1
!./utilities/run_warmup.sh {modelName} {batchSize}

Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 8
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 25
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Client: 
    Request count: 99
    Throughput: 197.935 infer/sec
    p50 latency: 40704 usec
    p85 latency: 40775 usec
    p90 latency: 40791 usec
    p95 latency: 40806 usec
    p99 latency: 40857 usec
    Avg gRPC time: 40703 usec ((un)marshal request/response 6 usec + response wait 40697 usec)
  Server: 
    Inference count: 792
    Execution count: 99
    Successful request count: 99
    Avg request latency: 40484 usec (overhead 70 usec + queue 56 usec + compute input 30 usec + compute infer 40318 usec + compute output 10 usec)

Inferences/Second vs. Client p85 Batch Latency
Concurrency: 1, throughput: 197.935 infer/sec, latency 40775 usec
Wa

In [16]:
%%time
modelName = "bertQA-onnx-trt-dynbatch"
maxConcurency= "10"
batchSize="1"
print("Running: "+modelName)

!./utilities/run_perf_analyzer_local.sh \
                    {modelName} \
                    {modelVersion} \
                    {precision} \
                    {batchSize} \
                    {maxLatency} \
                    {maxClientThreads} \
                    {maxConcurrency} \
                    {tritonServerHostName} \
                    {dockerBridge} \
                    {resultsFolderName} \
                    {profilingData} \
                    {measurement_request_count} \
                    {percentile_stability} \
                    {stability_percentage}

Running: bertQA-onnx-trt-dynbatch
Waiting for Triton Server to be ready at triton:8000...
200
Triton Server is ready!
*** Measurement Settings ***
  Batch size: 1
  Service Kind: Triton
  Using "count_windows" mode for stabilization
  Minimum number of samples in each window: 50
  Latency limit: 500 msec
  Concurrency limit: 10 concurrent requests
  Using synchronous calls for inference
  Stabilizing using p85 latency

Request concurrency: 1
  Pass [1] throughput: 86.9664 infer/sec. p85 latency: 11479 usec
  Pass [2] throughput: 87.9597 infer/sec. p85 latency: 11404 usec
  Pass [3] throughput: 87.9628 infer/sec. p85 latency: 11401 usec
  Client: 
    Request count: 263
    Throughput: 87.6296 infer/sec
    Avg client overhead: 0.01%
    Avg send request rate: 87.96 infer/sec
    p50 latency: 11367 usec
    p85 latency: 11421 usec
    p90 latency: 11440 usec
    p95 latency: 11493 usec
    p99 latency: 11625 usec
    Avg gRPC time: 11373 usec (marshal 2 usec + response wait 11370 usec +

지연 시간과 처리량 모두에서 상당한 개선이 보여야 합니다. 
* 바닐라 ONNX 구성 또는 바닐라 TorchScript와 비교할 때 그 영향은 얼마나 큽니까? 
* 여러 인스턴스를 구현할 때 병목 현상이 생긴 이유는 무엇이라고 생각하십니까?

강사와 결과를 논의하십시오.

<h3 style="color:green;">축하합니다!</h3><br>
다음을 사용하여 GPU 활용률을 개선하고 지연 시간을 줄이는 몇 가지 전략을 배웠습니다.

* 동시 모델 실행
* 스케줄링
* 동적 배치

이 다음 수업에서는 여러 동시 실행 수준에서 추론 성능을 보다 공식적으로 평가하고 체계적인 방식으로 추론 성능을 분석하는 방법에 대해 알아보겠습니다. 다음 노트북을 진행하십시오.<br>
[3.0 서버 성능](030_ServerPerformance.ipynb)

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>